# Australia COVID‑19 Weekly (Cases & Deaths) — Jan 2021 to Dec 2023
This minimal notebook downloads **daily** data from Our World in Data, then aggregates to **weekly sums**
(weeks ending **Sunday**) so you get a **complete weekly grid** from **2021‑01‑03** to **2023‑12‑31**.
It outputs `Australia_COVID_Weekly_2021_2023.csv`.

In [2]:
import pandas as pd

# -------- Config --------
COUNTRY = "Australia"
START_DATE = pd.Timestamp("2021-01-01")
END_DATE   = pd.Timestamp("2023-12-31")
WEEK_FREQ  = "W-SUN"  # week ends Sunday
OUT_CSV    = "Australia_COVID_Weekly_2021_2023.csv"

DAILY_URL = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"

# -------- Load DAILY, filter, clean --------
cols = ["location", "date", "new_cases", "new_deaths"]
daily = pd.read_csv(DAILY_URL, usecols=cols, parse_dates=["date"])
daily = daily.query("location == @COUNTRY").copy()

# Keep a 6-day buffer before START to compute the first full week ending after START
buffer_start = START_DATE - pd.Timedelta(days=6)
daily = daily[(daily["date"] >= buffer_start) & (daily["date"] <= END_DATE)].copy()

# Clean numeric
daily[["new_cases", "new_deaths"]] = (
    daily[["new_cases", "new_deaths"]]
    .apply(pd.to_numeric, errors="coerce")
    .fillna(0)
)

# -------- Weekly aggregation (sum) --------
weekly = (daily.set_index("date")
               .resample(WEEK_FREQ)
               .sum()[["new_cases", "new_deaths"]])

# Build a complete weekly index from the first Sunday in 2021 through 2023-12-31
first_week_end = pd.Timestamp("2021-01-03")  # first Sunday of 2021
last_week_end  = pd.Timestamp("2023-12-31")  # last Sunday of 2023
full_weeks = pd.date_range(first_week_end, last_week_end, freq=WEEK_FREQ)

# Reindex to enforce a complete weekly grid (fill missing with 0)
weekly = weekly.reindex(full_weeks, fill_value=0)
weekly.index.name = "week_end"

# Tidy columns
weekly = weekly.rename(columns={"new_cases": "weekly_cases", "new_deaths": "weekly_deaths"})
weekly["week_start"] = weekly.index - pd.to_timedelta(6, unit="D")
weekly = weekly.reset_index().rename(columns={"week_end": "week_end"})
weekly = weekly[["week_start", "week_end", "weekly_cases", "weekly_deaths"]]

# Save CSV
weekly.to_csv(OUT_CSV, index=False)

# Print all rows
pd.set_option("display.max_rows", None)
print(weekly)

# Summary
print("\nRows (full weekly coverage):", len(weekly))
print("Totals 2021–2023: cases =", weekly["weekly_cases"].sum(),
      ", deaths =", weekly["weekly_deaths"].sum())
print("Saved ->", OUT_CSV)


    week_start   week_end  weekly_cases  weekly_deaths
0   2020-12-28 2021-01-03         187.0            2.0
1   2021-01-04 2021-01-10         112.0            0.0
2   2021-01-11 2021-01-17         113.0            1.0
3   2021-01-18 2021-01-24          58.0            0.0
4   2021-01-25 2021-01-31          45.0            0.0
5   2021-02-01 2021-02-07          39.0            0.0
6   2021-02-08 2021-02-14          48.0            0.0
7   2021-02-15 2021-02-21          28.0            1.0
8   2021-02-22 2021-02-28          44.0            0.0
9   2021-03-01 2021-03-07          67.0            0.0
10  2021-03-08 2021-03-14          80.0            1.0
11  2021-03-15 2021-03-21          79.0            0.0
12  2021-03-22 2021-03-28          64.0            0.0
13  2021-03-29 2021-04-04          88.0            1.0
14  2021-04-05 2021-04-11          57.0            0.0
15  2021-04-12 2021-04-18         114.0            2.0
16  2021-04-19 2021-04-25         147.0            0.0
17  2021-0